# Chapter 8: Sensor Fusion

## Principles of Indoor Positioning and Indoor Navigation

---

### 📚 Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** the difference between loosely coupled (LC) and tightly coupled (TC) fusion
2. **Implement** IMU + UWB sensor fusion using Extended Kalman Filter
3. **Apply** chi-square gating for outlier rejection (Eqs. 8.8-8.9)
4. **Monitor** filter health using innovation analysis (Eqs. 8.5-8.6)
5. **Compare** fusion accuracy vs single-sensor positioning

### 📖 Book Reference

This notebook covers **Chapter 8: Sensor Fusion** with:
- **Section 8.2**: Tightly vs Loosely Coupled Architectures
- **Eq. (8.5)-(8.6)**: Innovation and innovation covariance
- **Eq. (8.8)-(8.9)**: Chi-square gating for outlier rejection

**Key Insight**: Sensor fusion combines the best of both worlds:
- **IMU**: High rate (100 Hz), smooth, but drifts
- **UWB**: Low rate (10 Hz), absolute position, but noisy

---


## 🚀 Setup

### Option A: Upload Repository ZIP (Recommended for Private Repos)

1. Download the repository as a ZIP file from your local machine
2. In Colab, click the folder icon on the left sidebar
3. Upload `IPIN_Book_Examples.zip` to `/content/`
4. Run the setup cell below

### Option B: Clone from GitHub (Public Repos)

If the repository is public, update the `GITHUB_REPO` variable below.


In [ ]:
# ========================================
# IPIN Book Examples - Chapter 8: Sensor Fusion
# ========================================

import os
import sys

# ============ CONFIGURATION ============
GITHUB_REPO = None  # e.g., "https://github.com/username/IPIN_Book_Examples.git"
# =======================================

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔍 Detecting setup method...")
    
    if os.path.exists('/content/IPIN_Book_Examples/core'):
        os.chdir('/content/IPIN_Book_Examples')
        print("✅ Repository already available.")
    elif os.path.exists('/content/IPIN_Book_Examples.zip'):
        print("📦 Found uploaded ZIP file, extracting...")
        import zipfile
        with zipfile.ZipFile('/content/IPIN_Book_Examples.zip', 'r') as zip_ref:
            zip_ref.extractall('/content/')
        if os.path.exists('/content/IPIN_Book_Examples-main'):
            os.rename('/content/IPIN_Book_Examples-main', '/content/IPIN_Book_Examples')
        os.chdir('/content/IPIN_Book_Examples')
        get_ipython().system('pip install -e . -q')
        print("✅ Setup from ZIP complete!")
    elif GITHUB_REPO:
        print(f"📥 Cloning from {GITHUB_REPO}...")
        get_ipython().system(f'git clone {GITHUB_REPO}')
        os.chdir('/content/IPIN_Book_Examples')
        get_ipython().system('pip install -e . -q')
        print("✅ Setup from GitHub complete!")
    else:
        print("❌ ERROR: Repository not found!")
        print("Upload 'IPIN_Book_Examples.zip' or set GITHUB_REPO variable.")
        raise FileNotFoundError("Repository not found.")
else:
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
    print(f"📂 Working directory: {os.getcwd()}")

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.stats import chi2

get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

# Import fusion modules
from core.estimators import ExtendedKalmanFilter
from core.fusion import (
    innovation,
    innovation_covariance,
    chi_square_gate,
    mahalanobis_distance_squared,
)

print("\n" + "="*60)
print("✅ Setup complete! Sensor fusion modules loaded.")
print("="*60)


---

# Part 1: Sensor Fusion Overview

## 1.1 Why Sensor Fusion?

Each sensor has strengths and weaknesses:

| Sensor | Strengths | Weaknesses |
|--------|-----------|------------|
| **IMU** | High rate (100Hz), smooth | Drifts unbounded |
| **UWB** | Absolute position, no drift | Noisy, low rate (10Hz) |
| **WiFi** | Ubiquitous | Low accuracy (1-5m) |
| **Camera** | Rich information | Computationally expensive |

**Fusion Goal**: Combine sensors to get the best of all worlds!

## 1.2 LC vs TC Architectures

| Aspect | **Loosely Coupled (LC)** | **Tightly Coupled (TC)** |
|--------|--------------------------|--------------------------|
| **Input** | Processed positions | Raw measurements (ranges) |
| **Updates** | 1 per epoch | N per epoch (one per anchor) |
| **Outlier handling** | All-or-nothing | Per-measurement rejection |
| **Complexity** | Lower | Higher |
| **Accuracy** | Good | Better |

## 1.3 The EKF Fusion Framework

```
┌─────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────┐
│   IMU   │ →  │   PREDICT   │ →  │   UPDATE    │ →  │ State   │
│ (100Hz) │    │ (propagate) │    │ (correct)   │    │ x, P    │
└─────────┘    └─────────────┘    └─────────────┘    └─────────┘
                                         ↑
                                   ┌─────────┐
                                   │   UWB   │
                                   │ (10Hz)  │
                                   └─────────┘
```


---

# Part 2: Simulating a Fusion Scenario

## 2.1 Scenario: 2D Robot Navigation

We'll simulate a robot moving in a 2D environment with:
- **IMU**: Accelerometer + gyroscope at 100 Hz
- **UWB**: Range measurements to 4 anchors at 10 Hz


In [ ]:
# ============================================
# Part 2: Simulate Sensor Fusion Scenario
# ============================================

np.random.seed(42)

# Simulation parameters
duration = 30.0  # seconds
imu_rate = 100   # Hz
uwb_rate = 10    # Hz

# UWB anchor positions (4 corners of a room)
uwb_anchors = np.array([
    [0, 0],
    [10, 0],
    [10, 10],
    [0, 10]
], dtype=float)
n_anchors = len(uwb_anchors)

# Generate ground truth trajectory (figure-8 pattern)
t_truth = np.arange(0, duration, 1/imu_rate)
n_steps = len(t_truth)

# Smooth trajectory: figure-8 centered at (5, 5)
omega = 2 * np.pi / duration
x_truth = 5 + 3 * np.sin(omega * t_truth)
y_truth = 5 + 2 * np.sin(2 * omega * t_truth)

# Velocities (derivatives)
vx_truth = 3 * omega * np.cos(omega * t_truth)
vy_truth = 4 * omega * np.cos(2 * omega * t_truth)

# Heading (from velocity)
yaw_truth = np.arctan2(vy_truth, vx_truth)

# Accelerations (for IMU)
ax_truth = -3 * omega**2 * np.sin(omega * t_truth)
ay_truth = -8 * omega**2 * np.sin(2 * omega * t_truth)
gyro_truth = np.gradient(yaw_truth, 1/imu_rate)

print(f"Generated trajectory:")
print(f"  Duration: {duration} s")
print(f"  IMU samples: {n_steps}")
print(f"  Path length: {np.sum(np.sqrt(np.diff(x_truth)**2 + np.diff(y_truth)**2)):.1f} m")

# Visualize setup
fig, ax = plt.subplots(figsize=(8, 8))
ax.plot(x_truth, y_truth, 'b-', linewidth=2, label='True trajectory')
ax.plot(x_truth[0], y_truth[0], 'go', markersize=12, label='Start')
ax.plot(x_truth[-1], y_truth[-1], 'r*', markersize=15, label='End')
ax.scatter(uwb_anchors[:, 0], uwb_anchors[:, 1], 
           s=200, c='orange', marker='^', edgecolors='black', 
           linewidths=2, label='UWB Anchors', zorder=5)
for i, (ax_, ay_) in enumerate(uwb_anchors):
    ax.annotate(f'A{i}', (ax_, ay_), textcoords="offset points", 
                xytext=(10, 10), fontsize=10, fontweight='bold')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('Fusion Scenario: IMU + UWB Navigation')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis('equal')
ax.set_xlim(-1, 12)
ax.set_ylim(-1, 12)
plt.tight_layout()
plt.show()


## 2.2 Generate Noisy Sensor Measurements

Now we add realistic noise to simulate real sensors:


In [ ]:
# ============================================
# Generate Noisy Sensor Measurements
# ============================================

# IMU noise parameters
sigma_accel = 0.5    # m/s² accelerometer noise
sigma_gyro = 0.05    # rad/s gyroscope noise
accel_bias = 0.1     # m/s² constant bias

# UWB noise parameters
sigma_uwb = 0.3      # m range measurement noise

# Generate noisy IMU measurements
accel_x_meas = ax_truth + accel_bias + sigma_accel * np.random.randn(n_steps)
accel_y_meas = ay_truth + sigma_accel * np.random.randn(n_steps)
gyro_z_meas = gyro_truth + sigma_gyro * np.random.randn(n_steps)

# Generate UWB range measurements (at lower rate)
uwb_indices = np.arange(0, n_steps, imu_rate // uwb_rate)  # 10 Hz
t_uwb = t_truth[uwb_indices]
n_uwb = len(uwb_indices)

# True ranges to each anchor
ranges_true = np.zeros((n_uwb, n_anchors))
for i, idx in enumerate(uwb_indices):
    pos = np.array([x_truth[idx], y_truth[idx]])
    ranges_true[i] = np.linalg.norm(uwb_anchors - pos, axis=1)

# Add noise to ranges
ranges_meas = ranges_true + sigma_uwb * np.random.randn(n_uwb, n_anchors)

print(f"Sensor measurements generated:")
print(f"  IMU: {n_steps} samples at {imu_rate} Hz")
print(f"  UWB: {n_uwb} epochs at {uwb_rate} Hz (4 ranges each)")
print(f"\nNoise levels:")
print(f"  Accelerometer: σ = {sigma_accel} m/s², bias = {accel_bias} m/s²")
print(f"  Gyroscope: σ = {sigma_gyro} rad/s")
print(f"  UWB ranging: σ = {sigma_uwb} m")


---

# Part 3: IMU-Only Dead Reckoning (Baseline)

First, let's see what happens with **IMU only** - no UWB corrections.

This demonstrates why sensor fusion is essential!


In [ ]:
# ============================================
# Part 3: IMU-Only Dead Reckoning
# ============================================

def imu_dead_reckoning(accel_x, accel_y, gyro_z, dt, x0, y0, vx0, vy0, yaw0):
    """Integrate IMU measurements to get position (no corrections).
    
    Args:
        accel_x, accel_y: Accelerometer measurements (m/s²)
        gyro_z: Gyroscope measurements (rad/s)
        dt: Time step (s)
        x0, y0: Initial position (m)
        vx0, vy0: Initial velocity (m/s)
        yaw0: Initial heading (rad)
    
    Returns:
        x, y, vx, vy, yaw: Integrated states
    """
    n = len(accel_x)
    x = np.zeros(n)
    y = np.zeros(n)
    vx = np.zeros(n)
    vy = np.zeros(n)
    yaw = np.zeros(n)
    
    x[0], y[0] = x0, y0
    vx[0], vy[0] = vx0, vy0
    yaw[0] = yaw0
    
    for i in range(1, n):
        # Update heading
        yaw[i] = yaw[i-1] + gyro_z[i-1] * dt
        
        # Rotate acceleration from body to world frame
        c, s = np.cos(yaw[i-1]), np.sin(yaw[i-1])
        ax_world = c * accel_x[i-1] - s * accel_y[i-1]
        ay_world = s * accel_x[i-1] + c * accel_y[i-1]
        
        # Simple integration (could use more sophisticated methods)
        vx[i] = vx[i-1] + ax_world * dt
        vy[i] = vy[i-1] + ay_world * dt
        x[i] = x[i-1] + vx[i-1] * dt + 0.5 * ax_world * dt**2
        y[i] = y[i-1] + vy[i-1] * dt + 0.5 * ay_world * dt**2
    
    return x, y, vx, vy, yaw

# Run IMU-only dead reckoning
dt = 1 / imu_rate
x_imu, y_imu, vx_imu, vy_imu, yaw_imu = imu_dead_reckoning(
    accel_x_meas, accel_y_meas, gyro_z_meas, dt,
    x_truth[0], y_truth[0], vx_truth[0], vy_truth[0], yaw_truth[0]
)

# Compute errors
error_imu = np.sqrt((x_imu - x_truth)**2 + (y_imu - y_truth)**2)

print("IMU-Only Dead Reckoning Results:")
print(f"  Final position error: {error_imu[-1]:.2f} m")
print(f"  Maximum error: {np.max(error_imu):.2f} m")
print(f"  Mean error: {np.mean(error_imu):.2f} m")

# Plot IMU-only results
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Trajectory
ax = axes[0]
ax.plot(x_truth, y_truth, 'b-', linewidth=2, label='Ground Truth')
ax.plot(x_imu, y_imu, 'r--', linewidth=2, label='IMU Only', alpha=0.8)
ax.scatter(uwb_anchors[:, 0], uwb_anchors[:, 1], 
           s=200, c='orange', marker='^', edgecolors='black', 
           linewidths=2, label='UWB Anchors', zorder=5)
ax.plot(x_truth[0], y_truth[0], 'go', markersize=12, label='Start')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('IMU-Only Dead Reckoning: Massive Drift!')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis('equal')

# Error over time
ax = axes[1]
ax.plot(t_truth, error_imu, 'r-', linewidth=2)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Position Error (m)')
ax.set_title('IMU Drift: Error Grows Unbounded')
ax.grid(True, alpha=0.3)
ax.axhline(y=1, color='g', linestyle='--', alpha=0.5, label='1m target')
ax.legend()

plt.tight_layout()
plt.show()

print("\n⚠️ IMU-only drift grows unbounded! This is why we need sensor fusion.")


---

# Part 4: Tightly Coupled EKF Fusion

## 4.1 State Vector and Models

**State vector**: x = [x, y, vx, vy, yaw] (position, velocity, heading)

**Process model** (IMU prediction): Integrates acceleration and angular rate

**Measurement model** (UWB range to anchor i): h(x) = distance to anchor

### Tightly Coupled = Raw ranges used directly in EKF update


In [ ]:
# ============================================
# Part 4: Tightly Coupled EKF Fusion
# ============================================

class TightlyCoupledEKF:
    """Tightly Coupled IMU + UWB Extended Kalman Filter.
    
    State: [x, y, vx, vy, yaw]
    """
    
    def __init__(self, anchors, sigma_accel, sigma_gyro, sigma_uwb):
        """Initialize TC-EKF.
        
        Args:
            anchors: UWB anchor positions (N, 2)
            sigma_accel: Accelerometer noise (m/s²)
            sigma_gyro: Gyroscope noise (rad/s)
            sigma_uwb: UWB ranging noise (m)
        """
        self.anchors = anchors
        self.n_anchors = len(anchors)
        
        # Process noise
        self.sigma_accel = sigma_accel
        self.sigma_gyro = sigma_gyro
        
        # Measurement noise
        self.R_uwb = sigma_uwb**2  # scalar for single range
        
        # State [x, y, vx, vy, yaw]
        self.x = np.zeros(5)
        
        # Covariance
        self.P = np.diag([1.0, 1.0, 1.0, 1.0, 0.1])
    
    def initialize(self, x0, y0, vx0, vy0, yaw0):
        """Set initial state."""
        self.x = np.array([x0, y0, vx0, vy0, yaw0])
    
    def predict(self, accel_x, accel_y, gyro_z, dt):
        """IMU prediction step.
        
        Args:
            accel_x, accel_y: Accelerometer measurements (body frame)
            gyro_z: Gyroscope measurement
            dt: Time step
        """
        x, y, vx, vy, yaw = self.x
        
        # Rotate acceleration to world frame
        c, s = np.cos(yaw), np.sin(yaw)
        ax_world = c * accel_x - s * accel_y
        ay_world = s * accel_x + c * accel_y
        
        # State prediction
        x_new = x + vx * dt + 0.5 * ax_world * dt**2
        y_new = y + vy * dt + 0.5 * ay_world * dt**2
        vx_new = vx + ax_world * dt
        vy_new = vy + ay_world * dt
        yaw_new = yaw + gyro_z * dt
        
        self.x = np.array([x_new, y_new, vx_new, vy_new, yaw_new])
        
        # Jacobian of process model
        F = np.array([
            [1, 0, dt, 0, 0],
            [0, 1, 0, dt, 0],
            [0, 0, 1, 0, 0],
            [0, 0, 0, 1, 0],
            [0, 0, 0, 0, 1]
        ])
        
        # Process noise
        Q = np.diag([
            0.5 * self.sigma_accel**2 * dt**4,
            0.5 * self.sigma_accel**2 * dt**4,
            self.sigma_accel**2 * dt**2,
            self.sigma_accel**2 * dt**2,
            self.sigma_gyro**2 * dt**2
        ])
        
        # Covariance prediction
        self.P = F @ self.P @ F.T + Q
    
    def range_measurement_model(self, anchor_idx):
        """Compute predicted range and Jacobian for one anchor.
        
        Args:
            anchor_idx: Index of the anchor
        
        Returns:
            h: Predicted range
            H: Jacobian (1, 5)
        """
        x, y = self.x[0], self.x[1]
        ax, ay = self.anchors[anchor_idx]
        
        dx = x - ax
        dy = y - ay
        r = np.sqrt(dx**2 + dy**2)
        
        # Jacobian: dh/dx
        H = np.zeros((1, 5))
        if r > 1e-6:
            H[0, 0] = dx / r
            H[0, 1] = dy / r
        
        return r, H
    
    def update_range(self, range_meas, anchor_idx, use_gating=True, gate_alpha=0.05):
        """Update with single range measurement.
        
        Args:
            range_meas: Measured range (m)
            anchor_idx: Index of the anchor
            use_gating: Whether to apply chi-square gating
            gate_alpha: Gating significance level
        
        Returns:
            accepted: Whether measurement was accepted
            nis: Normalized Innovation Squared
        """
        # Predicted measurement and Jacobian
        h_pred, H = self.range_measurement_model(anchor_idx)
        
        # Innovation
        y = range_meas - h_pred
        
        # Innovation covariance
        S = H @ self.P @ H.T + self.R_uwb
        S_scalar = S[0, 0]
        
        # NIS (Normalized Innovation Squared) = Mahalanobis distance
        nis = y**2 / S_scalar
        
        # Chi-square gating (1 DOF for scalar measurement)
        threshold = chi2.ppf(1 - gate_alpha, df=1)
        
        if use_gating and nis > threshold:
            return False, nis  # Reject outlier
        
        # Kalman gain
        K = self.P @ H.T / S_scalar
        
        # State update
        self.x = self.x + K.flatten() * y
        
        # Covariance update (Joseph form for stability)
        I_KH = np.eye(5) - K @ H
        self.P = I_KH @ self.P @ I_KH.T + K @ np.array([[self.R_uwb]]) @ K.T
        
        return True, nis

print("TightlyCoupledEKF class defined successfully!")


## 4.2 Run Tightly Coupled Fusion

Now we run the fusion with chi-square gating (Eq. 8.9) enabled:


In [ ]:
# ============================================
# Run Tightly Coupled EKF Fusion
# ============================================

# Create TC-EKF
tc_ekf = TightlyCoupledEKF(uwb_anchors, sigma_accel, sigma_gyro, sigma_uwb)
tc_ekf.initialize(x_truth[0], y_truth[0], vx_truth[0], vy_truth[0], yaw_truth[0])

# Storage for results
x_tc = np.zeros(n_steps)
y_tc = np.zeros(n_steps)
P_trace_tc = np.zeros(n_steps)
nis_history = []
gating_results = []

# Fusion loop
uwb_idx = 0
n_accepted = 0
n_rejected = 0

for i in range(n_steps):
    # IMU prediction (every step)
    tc_ekf.predict(accel_x_meas[i], accel_y_meas[i], gyro_z_meas[i], dt)
    
    # Check if UWB measurement available
    if uwb_idx < n_uwb and i == uwb_indices[uwb_idx]:
        # Process all 4 anchor ranges (TC = one update per anchor)
        for a_idx in range(n_anchors):
            r_meas = ranges_meas[uwb_idx, a_idx]
            accepted, nis = tc_ekf.update_range(r_meas, a_idx, use_gating=True)
            
            nis_history.append(nis)
            gating_results.append(accepted)
            
            if accepted:
                n_accepted += 1
            else:
                n_rejected += 1
        
        uwb_idx += 1
    
    # Store results
    x_tc[i] = tc_ekf.x[0]
    y_tc[i] = tc_ekf.x[1]
    P_trace_tc[i] = np.trace(tc_ekf.P[:2, :2])  # Position covariance trace

# Compute errors
error_tc = np.sqrt((x_tc - x_truth)**2 + (y_tc - y_truth)**2)

print("="*60)
print("Tightly Coupled EKF Fusion Results")
print("="*60)
print(f"\nMeasurement Processing:")
print(f"  UWB ranges accepted: {n_accepted}")
print(f"  UWB ranges rejected: {n_rejected}")
print(f"  Acceptance rate: {100*n_accepted/(n_accepted+n_rejected):.1f}%")
print(f"\nPosition Accuracy:")
print(f"  RMSE: {np.sqrt(np.mean(error_tc**2)):.3f} m")
print(f"  Max error: {np.max(error_tc):.3f} m")
print(f"  Final error: {error_tc[-1]:.3f} m")
print(f"\nImprovement over IMU-only:")
print(f"  RMSE reduction: {100*(1 - np.sqrt(np.mean(error_tc**2))/np.sqrt(np.mean(error_imu**2))):.1f}%")


In [ ]:
# ============================================
# Visualize TC Fusion Results
# ============================================

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. Trajectory comparison
ax = axes[0, 0]
ax.plot(x_truth, y_truth, 'b-', linewidth=2, label='Ground Truth')
ax.plot(x_imu, y_imu, 'r--', linewidth=1.5, alpha=0.5, label='IMU Only')
ax.plot(x_tc, y_tc, 'g-', linewidth=2, label='TC Fusion')
ax.scatter(uwb_anchors[:, 0], uwb_anchors[:, 1], 
           s=200, c='orange', marker='^', edgecolors='black', 
           linewidths=2, label='UWB Anchors', zorder=5)
ax.plot(x_truth[0], y_truth[0], 'ko', markersize=10, label='Start')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('Trajectory: TC Fusion vs IMU-Only')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis('equal')

# 2. Position error over time
ax = axes[0, 1]
ax.plot(t_truth, error_imu, 'r-', linewidth=1.5, alpha=0.5, label='IMU Only')
ax.plot(t_truth, error_tc, 'g-', linewidth=2, label='TC Fusion')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Position Error (m)')
ax.set_title('Position Error: Fusion Bounds the Drift!')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, min(50, max(error_imu) * 1.1))

# 3. NIS (Normalized Innovation Squared) plot
ax = axes[1, 0]
nis_array = np.array(nis_history)
gating_array = np.array(gating_results)
chi2_thresh = chi2.ppf(0.95, df=1)

ax.scatter(np.arange(len(nis_array))[gating_array], nis_array[gating_array], 
           c='green', s=10, alpha=0.5, label='Accepted')
ax.scatter(np.arange(len(nis_array))[~gating_array], nis_array[~gating_array], 
           c='red', s=10, alpha=0.5, label='Rejected')
ax.axhline(y=chi2_thresh, color='orange', linestyle='--', linewidth=2, 
           label=f'95% threshold ({chi2_thresh:.2f})')
ax.set_xlabel('Measurement Index')
ax.set_ylabel('NIS (Normalized Innovation Squared)')
ax.set_title('Innovation Monitoring: Chi-Square Gating')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(0, min(20, np.percentile(nis_array, 99) * 1.2))

# 4. Covariance trace over time
ax = axes[1, 1]
ax.plot(t_truth, np.sqrt(P_trace_tc), 'g-', linewidth=2)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Position Uncertainty (m)')
ax.set_title('Filter Uncertainty: Bounded by UWB Updates')
ax.grid(True, alpha=0.3)

# Mark UWB update times
for i, idx in enumerate(uwb_indices[::10]):  # Every 10th UWB epoch
    ax.axvline(x=t_truth[idx], color='orange', alpha=0.2, linewidth=1)

plt.tight_layout()
plt.show()


---

# Part 5: Chi-Square Gating (Eq. 8.8-8.9)

## 5.1 Theory: Innovation Monitoring

| Equation | Name | Formula | Purpose |
|----------|------|---------|---------|
| **8.5** | Innovation | y = z - h(x) | Measurement residual |
| **8.6** | Innovation Covariance | S = HPH' + R | Expected residual variance |
| **8.8** | NIS | d² = y'S⁻¹y | Normalized squared error |
| **8.9** | Gate | Accept if d² < threshold | Outlier rejection |

### Why Gating Matters

- **Without gating**: Outliers corrupt the filter, causing divergence
- **With gating**: Bad measurements are rejected, maintaining accuracy


In [ ]:
# ============================================
# Example: Gating with NLOS Outliers
# ============================================

# Add NLOS bias to some measurements (simulating multipath)
ranges_with_nlos = ranges_meas.copy()
nlos_probability = 0.2  # 20% of measurements have NLOS
nlos_bias = 2.0  # meters

# Randomly add NLOS bias
np.random.seed(123)
nlos_mask = np.random.random(ranges_with_nlos.shape) < nlos_probability
ranges_with_nlos[nlos_mask] += nlos_bias

print(f"Added NLOS bias ({nlos_bias}m) to {100*nlos_probability:.0f}% of measurements")
print(f"Total measurements with NLOS: {np.sum(nlos_mask)}")

# Run fusion WITHOUT gating
tc_ekf_no_gate = TightlyCoupledEKF(uwb_anchors, sigma_accel, sigma_gyro, sigma_uwb)
tc_ekf_no_gate.initialize(x_truth[0], y_truth[0], vx_truth[0], vy_truth[0], yaw_truth[0])

# Run fusion WITH gating
tc_ekf_with_gate = TightlyCoupledEKF(uwb_anchors, sigma_accel, sigma_gyro, sigma_uwb)
tc_ekf_with_gate.initialize(x_truth[0], y_truth[0], vx_truth[0], vy_truth[0], yaw_truth[0])

# Storage
x_no_gate = np.zeros(n_steps)
y_no_gate = np.zeros(n_steps)
x_with_gate = np.zeros(n_steps)
y_with_gate = np.zeros(n_steps)

uwb_idx = 0
n_accepted_gate = 0
n_rejected_gate = 0

for i in range(n_steps):
    # IMU prediction
    tc_ekf_no_gate.predict(accel_x_meas[i], accel_y_meas[i], gyro_z_meas[i], dt)
    tc_ekf_with_gate.predict(accel_x_meas[i], accel_y_meas[i], gyro_z_meas[i], dt)
    
    # UWB update
    if uwb_idx < n_uwb and i == uwb_indices[uwb_idx]:
        for a_idx in range(n_anchors):
            r_meas = ranges_with_nlos[uwb_idx, a_idx]
            
            # Without gating - accept all
            tc_ekf_no_gate.update_range(r_meas, a_idx, use_gating=False)
            
            # With gating
            accepted, _ = tc_ekf_with_gate.update_range(r_meas, a_idx, use_gating=True)
            if accepted:
                n_accepted_gate += 1
            else:
                n_rejected_gate += 1
        
        uwb_idx += 1
    
    x_no_gate[i] = tc_ekf_no_gate.x[0]
    y_no_gate[i] = tc_ekf_no_gate.x[1]
    x_with_gate[i] = tc_ekf_with_gate.x[0]
    y_with_gate[i] = tc_ekf_with_gate.x[1]

# Errors
error_no_gate = np.sqrt((x_no_gate - x_truth)**2 + (y_no_gate - y_truth)**2)
error_with_gate = np.sqrt((x_with_gate - x_truth)**2 + (y_with_gate - y_truth)**2)

print(f"\n{'='*60}")
print("Comparison: With vs Without Gating (NLOS scenario)")
print(f"{'='*60}")
print(f"\nWithout Gating:")
print(f"  RMSE: {np.sqrt(np.mean(error_no_gate**2)):.3f} m")
print(f"  Max error: {np.max(error_no_gate):.3f} m")
print(f"\nWith Gating:")
print(f"  RMSE: {np.sqrt(np.mean(error_with_gate**2)):.3f} m")
print(f"  Max error: {np.max(error_with_gate):.3f} m")
print(f"  Measurements rejected: {n_rejected_gate} ({100*n_rejected_gate/(n_accepted_gate+n_rejected_gate):.1f}%)")


In [ ]:
# ============================================
# Visualize Gating Impact
# ============================================

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Trajectory comparison
ax = axes[0]
ax.plot(x_truth, y_truth, 'b-', linewidth=2, label='Ground Truth')
ax.plot(x_no_gate, y_no_gate, 'r--', linewidth=2, alpha=0.7, label='No Gating')
ax.plot(x_with_gate, y_with_gate, 'g-', linewidth=2, label='With Gating')
ax.scatter(uwb_anchors[:, 0], uwb_anchors[:, 1], 
           s=200, c='orange', marker='^', edgecolors='black', 
           linewidths=2, label='UWB Anchors', zorder=5)
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('NLOS Scenario: Gating Saves the Day!')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis('equal')

# Error comparison
ax = axes[1]
ax.plot(t_truth, error_no_gate, 'r-', linewidth=2, alpha=0.7, label='No Gating')
ax.plot(t_truth, error_with_gate, 'g-', linewidth=2, label='With Gating')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Position Error (m)')
ax.set_title('Position Error: Gating Rejects NLOS Outliers')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Chi-square gating successfully rejects NLOS outliers!")


---

# Part 6: Loosely Coupled vs Tightly Coupled

## 6.1 Key Difference

| Aspect | LC | TC |
|--------|----|----|
| **Measurement input** | Position fix (x, y) | Raw ranges (r₁, r₂, r₃, r₄) |
| **EKF updates per epoch** | 1 | 4 (one per anchor) |
| **DOF for chi-square** | 2 | 1 |
| **Anchor dropout** | Fails if < 3 anchors | Graceful degradation |
| **Per-anchor outlier** | All-or-nothing | Individual rejection |


In [ ]:
# ============================================
# Part 6: Loosely Coupled Fusion
# ============================================

def solve_position_wls(ranges, anchors, sigma_range, x_init=None, max_iter=10):
    """Weighted Least Squares position solver from ranges.
    
    Args:
        ranges: Range measurements (N,)
        anchors: Anchor positions (N, 2)
        sigma_range: Range measurement noise
        x_init: Initial position guess (2,)
        max_iter: Maximum iterations
    
    Returns:
        position: Estimated position (2,)
        covariance: Position covariance (2, 2)
    """
    if x_init is None:
        x_init = np.mean(anchors, axis=0)
    
    x = x_init.copy()
    W = np.eye(len(ranges)) / sigma_range**2
    
    for _ in range(max_iter):
        # Predicted ranges
        d = anchors - x
        r_pred = np.linalg.norm(d, axis=1)
        
        # Jacobian
        H = -d / r_pred[:, np.newaxis]
        
        # Residual
        y = ranges - r_pred
        
        # WLS update
        HtWH = H.T @ W @ H
        HtWy = H.T @ W @ y
        
        try:
            dx = np.linalg.solve(HtWH, HtWy)
        except np.linalg.LinAlgError:
            break
        
        x = x + dx
        
        if np.linalg.norm(dx) < 1e-6:
            break
    
    # Covariance
    try:
        cov = np.linalg.inv(HtWH)
    except np.linalg.LinAlgError:
        cov = np.eye(2) * 100
    
    return x, cov


class LooselyCoupledEKF:
    """Loosely Coupled IMU + Position EKF."""
    
    def __init__(self, sigma_accel, sigma_gyro):
        self.sigma_accel = sigma_accel
        self.sigma_gyro = sigma_gyro
        self.x = np.zeros(5)
        self.P = np.diag([1.0, 1.0, 1.0, 1.0, 0.1])
    
    def initialize(self, x0, y0, vx0, vy0, yaw0):
        self.x = np.array([x0, y0, vx0, vy0, yaw0])
    
    def predict(self, accel_x, accel_y, gyro_z, dt):
        """Same IMU prediction as TC."""
        x, y, vx, vy, yaw = self.x
        
        c, s = np.cos(yaw), np.sin(yaw)
        ax_world = c * accel_x - s * accel_y
        ay_world = s * accel_x + c * accel_y
        
        self.x = np.array([
            x + vx * dt + 0.5 * ax_world * dt**2,
            y + vy * dt + 0.5 * ay_world * dt**2,
            vx + ax_world * dt,
            vy + ay_world * dt,
            yaw + gyro_z * dt
        ])
        
        F = np.array([
            [1, 0, dt, 0, 0],
            [0, 1, 0, dt, 0],
            [0, 0, 1, 0, 0],
            [0, 0, 0, 1, 0],
            [0, 0, 0, 0, 1]
        ])
        
        Q = np.diag([
            0.5 * self.sigma_accel**2 * dt**4,
            0.5 * self.sigma_accel**2 * dt**4,
            self.sigma_accel**2 * dt**2,
            self.sigma_accel**2 * dt**2,
            self.sigma_gyro**2 * dt**2
        ])
        
        self.P = F @ self.P @ F.T + Q
    
    def update_position(self, pos_meas, R_pos, use_gating=True, gate_alpha=0.05):
        """Update with position measurement (LC approach).
        
        Args:
            pos_meas: Measured position (2,)
            R_pos: Position measurement covariance (2, 2)
            use_gating: Whether to apply gating
            gate_alpha: Gating significance level
        
        Returns:
            accepted: Whether measurement was accepted
            nis: Normalized Innovation Squared
        """
        # Measurement model: h(x) = [x, y]
        H = np.array([
            [1, 0, 0, 0, 0],
            [0, 1, 0, 0, 0]
        ])
        
        # Innovation
        y = pos_meas - self.x[:2]
        
        # Innovation covariance
        S = H @ self.P @ H.T + R_pos
        
        # NIS
        try:
            S_inv = np.linalg.inv(S)
            nis = y @ S_inv @ y
        except np.linalg.LinAlgError:
            return False, 100.0
        
        # Chi-square gate (2 DOF for position)
        threshold = chi2.ppf(1 - gate_alpha, df=2)
        
        if use_gating and nis > threshold:
            return False, nis
        
        # Kalman gain
        K = self.P @ H.T @ S_inv
        
        # Update
        self.x = self.x + K @ y
        I_KH = np.eye(5) - K @ H
        self.P = I_KH @ self.P @ I_KH.T + K @ R_pos @ K.T
        
        return True, nis

print("LooselyCoupledEKF class defined successfully!")


In [ ]:
# ============================================
# Compare LC vs TC Fusion
# ============================================

# Reset TC EKF
tc_ekf_cmp = TightlyCoupledEKF(uwb_anchors, sigma_accel, sigma_gyro, sigma_uwb)
tc_ekf_cmp.initialize(x_truth[0], y_truth[0], vx_truth[0], vy_truth[0], yaw_truth[0])

# Create LC EKF
lc_ekf = LooselyCoupledEKF(sigma_accel, sigma_gyro)
lc_ekf.initialize(x_truth[0], y_truth[0], vx_truth[0], vy_truth[0], yaw_truth[0])

# Storage
x_tc_cmp = np.zeros(n_steps)
y_tc_cmp = np.zeros(n_steps)
x_lc = np.zeros(n_steps)
y_lc = np.zeros(n_steps)

uwb_idx = 0
tc_updates = 0
lc_updates = 0

for i in range(n_steps):
    # IMU prediction (both filters)
    tc_ekf_cmp.predict(accel_x_meas[i], accel_y_meas[i], gyro_z_meas[i], dt)
    lc_ekf.predict(accel_x_meas[i], accel_y_meas[i], gyro_z_meas[i], dt)
    
    # UWB update
    if uwb_idx < n_uwb and i == uwb_indices[uwb_idx]:
        ranges = ranges_meas[uwb_idx]
        
        # TC: Update with each range individually
        for a_idx in range(n_anchors):
            accepted, _ = tc_ekf_cmp.update_range(ranges[a_idx], a_idx, use_gating=True)
            if accepted:
                tc_updates += 1
        
        # LC: First solve for position, then update EKF
        pos_wls, cov_wls = solve_position_wls(
            ranges, uwb_anchors, sigma_uwb, 
            x_init=lc_ekf.x[:2]
        )
        accepted, _ = lc_ekf.update_position(pos_wls, cov_wls, use_gating=True)
        if accepted:
            lc_updates += 1
        
        uwb_idx += 1
    
    x_tc_cmp[i] = tc_ekf_cmp.x[0]
    y_tc_cmp[i] = tc_ekf_cmp.x[1]
    x_lc[i] = lc_ekf.x[0]
    y_lc[i] = lc_ekf.x[1]

# Compute errors
error_tc_cmp = np.sqrt((x_tc_cmp - x_truth)**2 + (y_tc_cmp - y_truth)**2)
error_lc = np.sqrt((x_lc - x_truth)**2 + (y_lc - y_truth)**2)

print("="*60)
print("LC vs TC Comparison (Clean Measurements)")
print("="*60)
print(f"\nTightly Coupled:")
print(f"  RMSE: {np.sqrt(np.mean(error_tc_cmp**2)):.3f} m")
print(f"  Max error: {np.max(error_tc_cmp):.3f} m")
print(f"  EKF updates: {tc_updates}")
print(f"\nLoosely Coupled:")
print(f"  RMSE: {np.sqrt(np.mean(error_lc**2)):.3f} m")
print(f"  Max error: {np.max(error_lc):.3f} m")
print(f"  EKF updates: {lc_updates}")


In [ ]:
# ============================================
# Visualize LC vs TC
# ============================================

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Trajectory
ax = axes[0]
ax.plot(x_truth, y_truth, 'b-', linewidth=2, label='Ground Truth')
ax.plot(x_tc_cmp, y_tc_cmp, 'g-', linewidth=2, label='Tightly Coupled')
ax.plot(x_lc, y_lc, 'm--', linewidth=2, label='Loosely Coupled')
ax.scatter(uwb_anchors[:, 0], uwb_anchors[:, 1], 
           s=200, c='orange', marker='^', edgecolors='black', 
           linewidths=2, label='UWB Anchors', zorder=5)
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('LC vs TC: Similar Performance (Clean Data)')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis('equal')

# Error comparison
ax = axes[1]
ax.plot(t_truth, error_tc_cmp, 'g-', linewidth=2, label='Tightly Coupled')
ax.plot(t_truth, error_lc, 'm--', linewidth=2, label='Loosely Coupled')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Position Error (m)')
ax.set_title('Position Error: LC vs TC')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 With clean data, LC and TC perform similarly.")
print("   TC shines when there are per-anchor outliers or dropouts!")


---

# Summary

## Key Takeaways

### 1. Sensor Fusion Benefits

| Approach | RMSE | Notes |
|----------|------|-------|
| **IMU Only** | Unbounded | Drifts without correction |
| **Fusion** | Bounded | UWB constrains drift |
| **Fusion + Gating** | Best | Rejects outliers |

### 2. LC vs TC Trade-offs

| Use Case | Recommended |
|----------|-------------|
| Simple, clean data | **LC** (simpler) |
| Per-anchor outliers | **TC** (granular rejection) |
| Anchor dropouts | **TC** (graceful degradation) |
| Computational limit | **LC** (fewer updates) |

### 3. Practical Tips

1. **Always use gating** - outliers happen in real environments
2. **Monitor NIS** - check if your filter is tuned correctly
3. **TC for production** - better robustness worth the complexity
4. **LC for prototyping** - faster development, similar accuracy

---

## Exercises

1. **Increase NLOS rate**: What happens when 50% of ranges have NLOS?
2. **Drop an anchor**: Simulate anchor 3 failing - how does TC handle it?
3. **Tune gating threshold**: What happens with α = 0.01 vs 0.10?

---

## Next Steps

Congratulations! You've completed all the core chapters:

| Chapter | Topic | Status |
|---------|-------|--------|
| Ch2 | Coordinate Systems | ✅ |
| Ch3 | State Estimation | ✅ |
| Ch4 | RF Positioning | ✅ |
| Ch5 | Fingerprinting | ✅ |
| Ch6 | Dead Reckoning | ✅ |
| Ch7 | SLAM | ✅ |
| Ch8 | Sensor Fusion | ✅ |

**You now have a comprehensive understanding of indoor positioning principles!**
